In [56]:
import pyupbit
import time
import json
import requests
import pandas as pd
import mysql.connector
from datetime import datetime, timedelta

# MySQL 서버에 연결
mydb = mysql.connector.connect(
    host="127.0.0.1",
    user="root",
    password="1234",
    database="deepCoin"
)

# 연결이 제대로 되었는지 확인
if mydb.is_connected():
    print("MySQL 데이터베이스에 성공적으로 연결되었습니다.")
else:
    print("MySQL 데이터베이스 연결에 실패했습니다.")

# 커서 생성
mycursor = mydb.cursor()

##### 종목마스터 수집

In [78]:
# 신규 종목마스터 조회
url = "https://api.upbit.com/v1/market/all?isDetails=true"
headers = {"accept": "application/json"}
response = requests.get(url, headers=headers)

mst_json = json.loads(response.text)
df = pd.DataFrame(mst_json)
df = df[['market', 'korean_name', 'english_name', 'market_warning']]
df = df[df['market'].str.contains('KRW')].reset_index(drop=True)

# 기존 마스터와 신규 호출 마스터 종목 비교
# SELECT 쿼리 실행
mycursor.execute("SELECT * FROM c_krw_mst")

# 결과 가져오기
result = mycursor.fetchall()

# 튜플의 0번째 인덱스만 가져와서 마켓 리스트 생성
market_list = [t[0] for t in result]

# KRW 종목 리스트 가져오기
# columns = pyupbit.get_tickers(fiat="KRW")

url = "https://api.upbit.com/v1/market/all?isDetails=true"
headers = {"accept": "application/json"}
response = requests.get(url, headers=headers)

mst_json = json.loads(response.text)
df = pd.DataFrame(mst_json)
df = df[['market', 'korean_name', 'english_name', 'market_warning']]
df = df[df['market'].str.contains('KRW')].reset_index(drop=True)


# DB생성용 Master 항목당 최대 길이 출력
max_length = df['market'].str.len().max()
max_string = df[df['market'].str.len() == max_length]['market'].iloc[0]
# print(max_string, len(max_string))

# 새로 호출한 종목리스트
mst_market_list = df['market'].tolist()

# 새로 상장된 종목 유무 확인
new_market = [x for x in mst_market_list if x not in market_list]

if(len(new_market) > 0):
    for new in new_market:
        # DataFrame을 튜플의 리스트로 변환
        # values = df.values.tolist() # 전체
        values = df[df['market'] == new].values.tolist()
        # 컬럼명 리스트
        columns = df.columns.tolist()

        # 삽입할 테이블 이름
        table_name = "c_krw_mst"

        # 데이터 삽입 쿼리 생성
        sql = "INSERT INTO {} ({}) VALUES ({})".format(
            table_name,
            ', '.join(columns),
            ', '.join(['%s'] * len(columns))
        )

        # 데이터 삽입
        mycursor.executemany(sql, values)

    # 변경 사항 커밋
    mydb.commit()
    print(f"신규 종목 : {new_market}")
    print(mycursor.rowcount, "레코드가 삽입되었습니다.")
    mydb.close()
else:
    print("신규 상장 종목 없음")
    pass

##### [전일데이터] 전일 일봉데이터 가져오기

In [51]:
# 현재 날짜와 시간을 구합니다.
today = datetime.now()

# timedelta를 사용하여 어제의 날짜를 구합니다.
yesterday = today - timedelta(days=1)
dayBeforeYesterday = today - timedelta(days=2)

# SELECT 쿼리 실행
mycursor.execute("SELECT * FROM c_krw_mst")

# 결과 가져오기
result = mycursor.fetchall()

# 튜플의 0번째 인덱스만 가져와서 마켓 리스트 생성
market_list = [t[0] for t in result]

# KRW 종목 리스트 가져오기
columns = ['date', 'market', 'open', 'high', 'low', 'close', 'volume', 'value']

# 삽입할 테이블 이름
table_name = "c_krw_day"

# 가져오기 실패한 종목 리스트
failed_market = []

# 어제 날짜 기준 일봉데이터 전체 호출
for market in market_list:
    df = pyupbit.get_ohlcv_from(ticker=market, interval="day", fromDatetime=dayBeforeYesterday,
                to=yesterday, period=0.5)
    date = yesterday.strftime('%Y-%m-%d')
    df['date'] = date
    df['market'] = market
    df = df[df['market'].str.contains('KRW')].reset_index(drop=True)
    df = df[['date', 'market', 'open', 'high', 'low', 'close', 'volume', 'value']]

    tuple_list = [tuple(x) for x in df.to_numpy()]

    try:
        # 데이터 삽입 쿼리 생성
        sql = "INSERT INTO {} ({}) VALUES ({})".format(
            table_name,
            ', '.join(columns),
            ', '.join(['%s'] * len(columns))
        )

        # 데이터 Insert
        mycursor.executemany(sql, tuple_list)
        print(mycursor.rowcount, f"{market} 레코드가 삽입되었습니다.", tuple_list)
    except:
        # 데이터 업데이트 쿼리 생성
        sql = "UPDATE {} SET {} WHERE date = '{}' AND market = '{}'".format(
            table_name,
            ', '.join(['{} = %s'.format(column) for column in columns]),
            date, 
            market
            )
        # 데이터 Update
        mycursor.executemany(sql, tuple_list)
        print(mycursor.rowcount, f"{market} 레코드가 업데이트되었습니다.", tuple_list)
    # api에서 df가 None으로 return 되는 경우가 있어 호출 시간 제어
    time.sleep(2)
# 변경 사항 커밋
mydb.commit()
print("commit")

0 KRW-1INCH 레코드가 업데이트되었습니다. [('2024-02-14', 'KRW-1INCH', 587.0, 610.4, 580.7, 609.0, 6359229.435555, 3805633416.199847)]
0 KRW-AAVE 레코드가 업데이트되었습니다. [('2024-02-14', 'KRW-AAVE', 121400.0, 124850.0, 120700.0, 123450.0, 51333.63096336, 6307899863.402336)]
0 KRW-ADA 레코드가 업데이트되었습니다. [('2024-02-14', 'KRW-ADA', 739.0, 788.2, 731.4, 781.7, 50661271.57118823, 38944260052.06487)]
0 KRW-AERGO 레코드가 업데이트되었습니다. [('2024-02-14', 'KRW-AERGO', 177.2, 181.0, 176.9, 180.3, 5017397.02188577, 899046129.0902808)]
0 KRW-AHT 레코드가 업데이트되었습니다. [('2024-02-14', 'KRW-AHT', 10.63, 10.72, 10.43, 10.71, 333466805.97076046, 3530158484.776159)]
0 KRW-ALGO 레코드가 업데이트되었습니다. [('2024-02-14', 'KRW-ALGO', 240.9, 249.4, 236.5, 248.9, 32790386.77555343, 8024422114.541644)]
0 KRW-ANKR 레코드가 업데이트되었습니다. [('2024-02-14', 'KRW-ANKR', 33.65, 36.9, 33.36, 35.93, 299101402.9393296, 10560343292.381548)]
0 KRW-APT 레코드가 업데이트되었습니다. [('2024-02-14', 'KRW-APT', 12480.0, 12950.0, 12280.0, 12810.0, 1071104.13488292, 13562517814.995716)]
0 KRW-AQT 레코

##### [과거데이터] From - To 일봉데이터 가져오기

In [80]:
# datetime 객체 생성
from_day = datetime(2024, 2, 16, 0, 0, 0)
to_day = datetime(2024, 2, 19, 0, 0, 0)

# SELECT 쿼리 실행
mycursor.execute("SELECT * FROM c_krw_mst")

# 결과 가져오기
result = mycursor.fetchall()

# 튜플의 0번째 인덱스만 가져와서 마켓 리스트 생성
market_list = [t[0] for t in result]

# KRW 종목 리스트 가져오기
columns = ['date', 'market', 'open', 'high', 'low', 'close', 'volume', 'value']

# 삽입할 테이블 이름
table_name = "c_krw_day"

# 가져오기 실패한 종목 리스트
failed_market = []

# 어제 날짜 기준 일봉데이터 전체 호출
for market in market_list:
    df = pyupbit.get_ohlcv_from(ticker=market, interval="day", fromDatetime=from_day,
                to=to_day, period=0.5)
    df['date'] = df.index.strftime('%Y-%m-%d')
    df['market'] = market
    df = df[['date', 'market', 'open', 'high', 'low', 'close', 'volume', 'value']]

    tuple_list = [tuple(x) for x in df.to_numpy()]

    try:
        # 데이터 삽입 쿼리 생성
        sql = "INSERT INTO {} ({}) VALUES ({})".format(
            table_name,
            ', '.join(columns),
            ', '.join(['%s'] * len(columns))
        )

        # 데이터 Insert
        mycursor.executemany(sql, tuple_list)
        print(mycursor.rowcount, f"{market} 레코드가 삽입되었습니다.", tuple_list)
    except:
        # 데이터 업데이트 쿼리 생성
        sql = "UPDATE {} SET {} WHERE date = '{}' AND market = '{}'".format(
            table_name,
            ', '.join(['{} = %s'.format(column) for column in columns]),
            date, 
            market
            )
        # 데이터 Update
        mycursor.executemany(sql, tuple_list)
        print(mycursor.rowcount, f"{market} 레코드가 업데이트되었습니다.", tuple_list)
    # api에서 df가 None으로 return 되는 경우가 있어 호출 시간 제어
    time.sleep(2)
# 변경 사항 커밋
mydb.commit()
print("commit")

NameError: name 'date' is not defined

##### [과거데이터] 최초 종목별 일봉데이터 가져오기

In [49]:
# SELECT 쿼리 실행
mycursor.execute("SELECT * FROM c_krw_mst")

# 결과 가져오기
result = mycursor.fetchall()

# 튜플의 0번째 인덱스만 가져와서 마켓 리스트 생성
market_list = [t[0] for t in result]

# KRW 종목 리스트 가져오기
columns = ['date', 'market', 'open', 'high', 'low', 'close', 'volume', 'value']

# 현재 날짜와 시간을 구합니다.
today = datetime.now()

# timedelta를 사용하여 어제의 날짜를 구합니다.
yesterday = today - timedelta(days=1)

# 삽입할 테이블 이름
table_name = "c_krw_day"

# 어제 날짜 기준 일봉데이터 전체 호출
for market in market_list:
    df = pyupbit.get_ohlcv_from(ticker=market, interval="day", fromDatetime=None,
                   to=yesterday, period=0.5)
    df['date'] = df.index.strftime('%Y-%m-%d')
    df['market'] = market
    df = df[['date', 'market', 'open', 'high', 'low', 'close', 'volume', 'value']]

    tuple_list = [tuple(x) for x in df.to_numpy()]

    # 데이터 삽입 쿼리 생성
    sql = "INSERT INTO {} ({}) VALUES ({})".format(
        table_name,
        ', '.join(columns),
        ', '.join(['%s'] * len(columns))
    )
    # 데이터 삽입
    mycursor.executemany(sql, tuple_list)
    print(mycursor.rowcount, f"{market} 레코드가 삽입되었습니다.")

# 변경 사항 커밋
mydb.commit()
print("전체 레코드가 삽입되었습니다.")

IntegrityError: 1062 (23000): Duplicate entry '2021-10-27-KRW-1INCH' for key 'PRIMARY'